In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
from skimage.io import imread 
from PIL import Image
import csv
import torch
import copy
import torchvision
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
from torch.utils import data
import torchvision.transforms as transf
import torch.nn.functional as F
from torchvision.transforms.transforms import RandomAdjustSharpness
import torch.nn as nn
import random
import cv2
import math
import os
from torch import optim
from torch.nn.modules.conv import ConvTranspose2d
from torch.autograd import Function
import glob

In [ ]:
# Hyperparameter Tuning
epoch_n = 50
l_rate = 0.0001
batch_size_tr = 15
batch_size_val = 15

### Structure of dataset

--- /train
------/class_1
---------/mask_gen1.png
---------/mask_gen2.png

------/class_2
---------/mask_gen3.png
---------/mask_gen4.png

------/class_3
...

--- /val
------/class_1
---------/mask_gen6.png
---------/mask_gen7.png

------/class_2
---------/mask_gen8.png
---------/mask_gen9.png

------/class_3
...

In [ ]:
# Augmentation of dataset

train_transform = transf.Compose([
    transf.Resize((380,380)),
    transf.RandomHorizontalFlip(p=0.5),
    transf.RandomRotation(degrees=45),
    transf.RandomVerticalFlip(p=0.5),
    transf.ColorJitter(brightness=0.5),
    transf.RandomAdjustSharpness(sharpness_factor=5),
    transf.RandomAutocontrast(),
    transf.ToTensor()
])

val_transform = transf.Compose([
    transf.Resize((224,224)),
    transf.ToTensor()
])
train_ds = ImageFolder('/content/drive/MyDrive/PH2/train_classifier/', transform=train_transform)
val_ds = ImageFolder('/content/drive/MyDrive/PH2/val_classifier/', transform=val_transform)
#test_ds = ImageFolder('/content/drive/MyDrive/Lung_Carcinoma/data_folder/test', transform=val_transform)
train_load = DataLoader(dataset=train_ds, batch_size=batch_size_tr, shuffle=True, num_workers=2, drop_last=False)
val_load = DataLoader(dataset=val_ds, batch_size=batch_size_val, shuffle=True, num_workers=2, drop_last=False)
#test_load = DataLoader(dataset=test_ds, batch_size=batch_size_val, shuffle=False, drop_last=False)
if torch.cuda.is_available():
  device='cuda'
else:
  device='cpu'

In [ ]:
# Deep Feature Extraction modifications

class refined(nn.Module):
  def __init__(self, out_classes=3):
    super(refined,self).__init__()
    eff_model = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_efficientnet_b4', pretrained=True)
    self.initial = nn.Sequential(*list(eff_model.children())[:1])
    self.ref_model_freeze = nn.Sequential(*list(eff_model.children())[1][:6])
    self.ref_model = nn.Sequential(*list(eff_model.children())[1][6],
                                   *list(eff_model.children())[2:][0])
    self.pool = nn.AdaptiveAvgPool2d(1)
    self.flat = nn.Flatten()
    # self.pool = nn.AdaptiveAvgPool2d(output_size=1)
    # self.flat = nn.Flatten()
    self.fcc = nn.Sequential(
                 nn.Linear(1792,out_classes))
  
  def forward(self,images):
    raw_featr_freeze = self.ref_model_freeze(self.initial(images))
    raw_featr = self.flat(self.pool(self.ref_model(raw_featr_freeze)))
    output = self.fcc(raw_featr)
    return raw_featr,output

In [ ]:
##### training #####

def train(model, criterion_tr, criterion_val, optim, epoch_n):
  best_loss=5 # ekhane previous best acc ta dibi in the format 0.XX
  train_loss_list = []
  val_loss_list = []
  best_featr_tr = []
  best_ver_labels_ext_tr = []
  best_featr_val = []
  best_ver_labels_ext_val = []
  #best_model_wts = copy.deepcopy(model.state_dict())
  for epoch in range(epoch_n):
    featr_tensor_tr = np.zeros((1,1792))
    labels_ext_tr = []
    model.train()
    running_train_loss = 0.0
    running_train_acc = 0.0
    for images,labels in train_load:
      images = images.to(device)
      labels = labels.to(device)
      with torch.set_grad_enabled(True):
        featr,output = model(images)
        featr_tensor_tr = np.append(featr_tensor_tr,featr.cpu().detach().numpy(), axis=0)
        labels_ext_tr = np.append(labels_ext_tr,labels.cpu().detach().numpy(),axis=0)
        _,pred = torch.max(output,1)
        loss = criterion_tr(output,labels,featr)
        loss.backward()
        optim.step()
      optim.zero_grad()
      running_train_loss += loss.item()*batch_size_tr
      running_train_acc += torch.sum(pred==labels)
    running_val_loss, running_val_acc, featr_tensor_val, labels_ext_val = eval(model, criterion_val)
    epoch_train_loss = running_train_loss/len(train_ds)
    epoch_train_acc = running_train_acc.double()/len(train_ds)
    print("Epoch: {}".format(epoch+1))
    print('-'*10)
    print('Train Loss: {:.4f}   Train Acc: {:.4f}'.format(epoch_train_loss,epoch_train_acc))
    epoch_val_loss = running_val_loss/len(val_ds)
    epoch_val_acc = running_val_acc.double()/len(val_ds)
    print('Val Loss: {:.4f}   Val Acc: {:.4f}'.format(epoch_val_loss,epoch_val_acc))
    print('\n')
    if best_loss > epoch_val_loss:
      best_loss = epoch_val_loss
      best_model_wts = copy.deepcopy(model.state_dict())
      state={
          "model_state":model.state_dict(),
          "optimizer_state":optim.state_dict(),
            }
      torch.save(state,'/content/drive/MyDrive/PH2/model_wts/model_wts_effnet_PH2_run1.pth') # ekhane notun weight er src ta korbi 
      best_featr_tr = featr_tensor_tr
      best_ver_labels_ext_tr = labels_ext_tr
      best_featr_val = featr_tensor_val
      best_ver_labels_ext_val = labels_ext_val

    last_model_wts = copy.deepcopy(model.state_dict())
    state_1={
        "model_state":model.state_dict(),
        "optimizer_state":optim.state_dict(),
          }
    torch.save(state_1,'/content/drive/MyDrive/PH2/model_wts/model_wts_effnet_PH2_run1_last.pth')
    train_loss_list = train_loss_list + [epoch_train_loss]
    val_loss_list = val_loss_list + [epoch_val_loss]
  model = model.load_state_dict(best_model_wts)
  print("The model with the best performance has a loss of :{:.4f}".format(best_loss))
  return model, best_featr_tr, best_ver_labels_ext_tr, best_featr_val, best_ver_labels_ext_val, train_loss_list, val_loss_list


def eval(model, criterion_val):
  model.eval()
  featr_tensor_val = np.zeros((1,1792))
  labels_ext_val = []
  running_val_loss = 0.0
  running_val_acc = 0.0
  for images,labels in val_load:
    images = images.to(device)
    labels = labels.to(device)
    featr,output = model(images)
    featr_tensor_val = np.append(featr_tensor_val,featr.cpu().detach().numpy(), axis =0)
    labels_ext_val = np.append(labels_ext_val,labels.cpu().detach().numpy(),axis=0)
    _,pred = torch.max(output,1)
    loss = criterion_val(output,labels,featr)
    running_val_loss += loss.item()*batch_size_val
    running_val_acc += torch.sum(pred==labels)
  return running_val_loss, running_val_acc, featr_tensor_val, labels_ext_val

In [ ]:
# samples in each class for train and val

cls_num_list_train=[57, 63, 32]
cls_num_list_val=[23, 17, 8]

In [ ]:
# Defining per class weights for CBReweight for train

train_sampler = None
beta = 0.9999
effective_num = 1.0 - np.power(beta, cls_num_list_train)
per_cls_weights = (1.0 - beta) / np.array(effective_num)
per_cls_weights = per_cls_weights / np.sum(per_cls_weights) * len(cls_num_list_train)
per_cls_weights_tr = torch.FloatTensor(per_cls_weights).to(device)

In [ ]:
# Defining per class weights for CBReweight for validation

train_sampler = None
beta = 0.9999
effective_num = 1.0 - np.power(beta, cls_num_list_val)
per_cls_weights = (1.0 - beta) / np.array(effective_num)
per_cls_weights = per_cls_weights / np.sum(per_cls_weights) * len(cls_num_list_val)
per_cls_weights_val = torch.FloatTensor(per_cls_weights).to(device)

In [ ]:
num_classes = 3
# IB_Focal Loss 
def ib_focal_loss(input_values, ib, gamma):
    """Computes the ib focal loss"""
    p = torch.exp(-input_values)
    loss = (1 - p) ** gamma * input_values * ib
    return loss.mean()

class IB_FocalLoss(nn.Module):
    def __init__(self, weight, alpha=1000000, gamma=0):
        super(IB_FocalLoss, self).__init__()
        assert alpha > 0
        self.alpha = alpha
        self.epsilon = 0.001
        self.weight = weight
        self.gamma = gamma

    def forward(self, input, target, features):
        grads = torch.sum(torch.abs(F.softmax(input, dim=1) - F.one_hot(target, num_classes)),1) # N * 1
        # print(grads.shape)
        ib = grads*(torch.sum(features.reshape(-1)))
        ib = self.alpha / (ib + self.epsilon)
        return ib_focal_loss(F.cross_entropy(input, target, reduction='none', weight=self.weight), ib, self.gamma)

In [ ]:
model = refined()
# state=torch.load('/content/drive/MyDrive/PH2/model_wts/model_wts_effnet_PH2_run1_last.pth') # ekhane previous weight ta src korbi 
# model.load_state_dict(state['model_state'])
model = model.to(device)
criterion_tr = IB_FocalLoss(per_cls_weights_tr)
criterion_tr=criterion_tr.to(device)
criterion_val = IB_FocalLoss(per_cls_weights_val)
criterion_val=criterion_val.to(device)
optim = torch.optim.Adam(model.parameters(), l_rate)
# optim.load_state_dict(state['optimizer_state'])

In [ ]:
model, best_featr_tr, labels_tr, best_featr_val, labels_val, train_loss_list, val_loss_list = train(model, criterion_tr, criterion_val, optim, epoch_n)